# 3.2 Image Captioning with LSTMs
在之前的3.1练习中，你实现了一个普通的 RNN 并将其应用于图像描述生成。在本作业中，你将实现 LSTM 更新规则并将其用于图像描述生成。

In [ ]:
# Setup cell.
import time, os, json
import numpy as np
import matplotlib.pyplot as plt

from mml.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from mml.rnn_layers import *
from mml.captioning_solver import CaptioningSolver
from mml.classifiers.rnn import CaptioningRNN
from mml.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions
from mml.image_utils import image_from_url

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # Set default size of plots.
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# COCO Dataset
As in the previous notebook, we will use the COCO dataset for captioning.

In [ ]:
# Load COCO data from disk into a dictionary.
data = load_coco_data(pca_features=True)

# Print out all the keys and values from the data dictionary.
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

# LSTM

普通 RNN 的一种常见变体是长短期记忆网络（LSTM）。普通 RNN 在长序列上训练可能会由于重复矩阵乘法导致梯度消失或爆炸而变得困难。LSTM 通过用门控机制替代普通 RNN 的简单更新规则来解决这个问题。

与普通 RNN 类似，在每个时间步我们接收一个输入 $x_t\in\mathbb{R}^D$ 和前一个隐藏状态 $h_{t-1}\in\mathbb{R}^H$；此外，LSTM 还维护一个 $H$ 维的 *单元状态*，因此我们还接收前一个单元状态 $c_{t-1}\in\mathbb{R}^H$。LSTM 的可学习参数包括一个 *输入到隐藏* 矩阵 $W_x\in\mathbb{R}^{4H\times D}$，一个 *隐藏到隐藏* 矩阵 $W_h\in\mathbb{R}^{4H\times H}$，以及一个 *偏置向量* $b\in\mathbb{R}^{4H}$。

在每个时间步，我们首先计算一个 *激活向量* $a\in\mathbb{R}^{4H}$，计算公式为 $a=W_xx_t + W_hh_{t-1}+b$。然后我们将其分为四个向量 $a_i,a_f,a_o,a_g\in\mathbb{R}^H$，其中 $a_i$ 是 $a$ 的前 $H$ 个元素，$a_f$ 是接下来的 $H$ 个元素，依此类推。接着我们计算 *输入门* $i\in\mathbb{R}^H$，*遗忘门* $f\in\mathbb{R}^H$，*输出门* $o\in\mathbb{R}^H$，以及 *块输入* $g\in\mathbb{R}^H$，公式如下：

$$
i = \sigma(a_i) \hspace{2pc}
f = \sigma(a_f) \hspace{2pc}
o = \sigma(a_o) \hspace{2pc}
g = \tanh(a_g)
$$

其中 $\sigma$ 是逐元素的 sigmoid 函数，$\tanh$ 是逐元素的双曲正切函数。

最后，我们计算下一个单元状态 $c_t$ 和下一个隐藏状态 $h_t$：

$$
c_{t} = f\odot c_{t-1} + i\odot g \hspace{4pc}
h_t = o\odot\tanh(c_t)
$$

其中 $\odot$ 是向量的逐元素乘积。

在接下来的部分中，我们将实现 LSTM 更新规则并将其应用于图像描述生成任务。

在代码中，我们假设数据以批量形式存储，因此 $X_t \in \mathbb{R}^{N\times D}$，并且会使用 *转置* 版本的参数：$W_x \in \mathbb{R}^{D \times 4H}$，$W_h \in \mathbb{R}^{H\times 4H}$，以便通过以下公式高效地计算激活 $A \in \mathbb{R}^{N\times 4H}$：

$$
A = X_t W_x + H_{t-1} W_h
$$

# LSTM: 前向步骤

在文件 `mml/rnn_layers.py` 中的 `lstm_step_forward` 函数中实现 LSTM 单个时间步的前向传播。此过程应与之前实现的 `rnn_step_forward` 函数类似，但需要使用 LSTM 的更新规则。

完成后，运行以下代码以对你的实现进行简单测试。你应该看到误差在 `e-8` 量级或更小。

In [ ]:
N, D, H = 3, 4, 5
x = np.linspace(-0.4, 1.2, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.3, 0.7, num=N*H).reshape(N, H)
prev_c = np.linspace(-0.4, 0.9, num=N*H).reshape(N, H)
Wx = np.linspace(-2.1, 1.3, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.7, 2.2, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.3, 0.7, num=4*H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

expected_next_h = np.asarray([
    [ 0.24635157,  0.28610883,  0.32240467,  0.35525807,  0.38474904],
    [ 0.49223563,  0.55611431,  0.61507696,  0.66844003,  0.7159181 ],
    [ 0.56735664,  0.66310127,  0.74419266,  0.80889665,  0.858299  ]])
expected_next_c = np.asarray([
    [ 0.32986176,  0.39145139,  0.451556,    0.51014116,  0.56717407],
    [ 0.66382255,  0.76674007,  0.87195994,  0.97902709,  1.08751345],
    [ 0.74192008,  0.90592151,  1.07717006,  1.25120233,  1.42395676]])

print('next_h error: ', rel_error(expected_next_h, next_h))
print('next_c error: ', rel_error(expected_next_c, next_c))

# LSTM: 反向步骤

在文件 `mml/rnn_layers.py` 中的 `lstm_step_backward` 函数中实现 LSTM 单个时间步的反向传播。完成后，运行以下代码以对你的实现进行数值梯度检查。你应该看到误差在 `e-7` 量级或更小。

In [ ]:
np.random.seed(231)

N, D, H = 4, 5, 6
x = np.random.randn(N, D)
prev_h = np.random.randn(N, H)
prev_c = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)

fx_h = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fh_h = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fc_h = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWx_h = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWh_h = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fb_h = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]

fx_c = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fh_c = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fc_c = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWx_c = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWh_c = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fb_c = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx_h, x, dnext_h) + num_grad(fx_c, x, dnext_c)
dh_num = num_grad(fh_h, prev_h, dnext_h) + num_grad(fh_c, prev_h, dnext_c)
dc_num = num_grad(fc_h, prev_c, dnext_h) + num_grad(fc_c, prev_c, dnext_c)
dWx_num = num_grad(fWx_h, Wx, dnext_h) + num_grad(fWx_c, Wx, dnext_c)
dWh_num = num_grad(fWh_h, Wh, dnext_h) + num_grad(fWh_c, Wh, dnext_c)
db_num = num_grad(fb_h, b, dnext_h) + num_grad(fb_c, b, dnext_c)

dx, dh, dc, dWx, dWh, db = lstm_step_backward(dnext_h, dnext_c, cache)

print('dx error: ', rel_error(dx_num, dx))
print('dh error: ', rel_error(dh_num, dh))
print('dc error: ', rel_error(dc_num, dc))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# LSTM: 前向传播

在文件 `mml/rnn_layers.py` 中的 `lstm_forward` 函数中，实现 `lstm_forward` 以对整个时间序列的数据执行 LSTM 的前向传播。

完成后，运行以下代码以检查你的实现。你应该看到误差在 `e-7` 量级或更小。

In [ ]:
N, D, H, T = 2, 5, 4, 3
x = np.linspace(-0.4, 0.6, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.4, 0.8, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.9, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.3, 0.6, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.2, 0.7, num=4*H)

h, cache = lstm_forward(x, h0, Wx, Wh, b)

expected_h = np.asarray([
 [[ 0.01764008,  0.01823233,  0.01882671,  0.0194232 ],
  [ 0.11287491,  0.12146228,  0.13018446,  0.13902939],
  [ 0.31358768,  0.33338627,  0.35304453,  0.37250975]],
 [[ 0.45767879,  0.4761092,   0.4936887,   0.51041945],
  [ 0.6704845,   0.69350089,  0.71486014,  0.7346449 ],
  [ 0.81733511,  0.83677871,  0.85403753,  0.86935314]]])

print('h error: ', rel_error(expected_h, h))

# LSTM: 反向传播

在文件 `mml/rnn_layers.py` 中的 `lstm_backward` 函数中，实现 LSTM 对整个时间序列数据的反向传播。完成后，运行以下代码以对你的实现进行数值梯度检查。你应该看到误差在 `e-8` 量级或更小。（对于 `dWh`，如果误差在 `e-6` 量级或更小，也是可以接受的）。

In [ ]:
from mml.rnn_layers import lstm_forward, lstm_backward
np.random.seed(231)

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

out, cache = lstm_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = lstm_backward(dout, cache)

fx = lambda x: lstm_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: lstm_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: lstm_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: lstm_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: lstm_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print('dx error: ', rel_error(dx_num, dx))
print('dh0 error: ', rel_error(dh0_num, dh0))
print('dWx error: ', rel_error(dWx_num, dWx))
print('dWh error: ', rel_error(dWh_num, dWh))
print('db error: ', rel_error(db_num, db))

# LSTM 描述生成模型

现在你已经实现了 LSTM，接下来需要你更新文件 `mml/classifiers/rnn.py` 中 `CaptioningRNN` 类的 `loss` 方法，以处理 `self.cell_type` 为 `lstm` 的情况。这部分更新应该需要少于 10 行代码。

完成后，运行以下代码以检查你的实现。你应该看到误差在 `e-10` 量级或更小。

In [ ]:
N, D, W, H = 10, 20, 30, 40
word_to_idx = {'<NULL>': 0, 'cat': 2, 'dog': 3}
V = len(word_to_idx)
T = 13

model = CaptioningRNN(
    word_to_idx,
    input_dim=D,
    wordvec_dim=W,
    hidden_dim=H,
    cell_type='lstm',
    dtype=np.float64
)

# Set all model parameters to fixed values
for k, v in model.params.items():
  model.params[k] = np.linspace(-1.4, 1.3, num=v.size).reshape(*v.shape)

features = np.linspace(-0.5, 1.7, num=N*D).reshape(N, D)
captions = (np.arange(N * T) % V).reshape(N, T)

loss, grads = model.loss(features, captions)
expected_loss = 9.82445935443

print('loss: ', loss)
print('expected loss: ', expected_loss)
print('difference: ', abs(loss - expected_loss))

# 在小数据集上过拟合 LSTM 描述生成模型

运行以下代码以在之前用于 RNN 的相同小数据集上对 LSTM 描述生成模型进行过拟合。你应该看到最终的损失小于 0.5。

In [ ]:
np.random.seed(231)

small_data = load_coco_data(max_train=50)

small_lstm_model = CaptioningRNN(
    cell_type='lstm',
    word_to_idx=data['word_to_idx'],
    input_dim=data['train_features'].shape[1],
    hidden_dim=512,
    wordvec_dim=256,
    dtype=np.float32,
)

small_lstm_solver = CaptioningSolver(
    small_lstm_model, small_data,
    update_rule='adam',
    num_epochs=50,
    batch_size=25,
    optim_config={
     'learning_rate': 5e-3,
    },
    lr_decay=0.995,
    verbose=True, print_every=10,
)

small_lstm_solver.train()

# Plot the training losses
plt.plot(small_lstm_solver.loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training loss history')
plt.show()

打印最终的训练损失。你应该看到最终的损失小于 0.5。

In [ ]:
print('Final loss: ', small_lstm_solver.loss_history[-1])

# 测试时的 LSTM 采样

修改 `CaptioningRNN` 类的 `sample` 方法以处理 `self.cell_type` 为 `lstm` 的情况。这部分代码修改应少于 10 行。

完成后，运行以下代码以从过拟合的 LSTM 模型中对一些训练集和验证集样本进行采样。与 RNN 类似，训练集的结果应该非常好，而验证集的结果可能不会很合理（因为我们正在过拟合）。

In [ ]:
# If you get an error, the URL just no longer exists, so don't worry!
# You can re-sample as many times as you want.
for split in ['train', 'val']:
    minibatch = sample_coco_minibatch(small_data, split=split, batch_size=2)
    gt_captions, features, urls = minibatch
    gt_captions = decode_captions(gt_captions, data['idx_to_word'])

    sample_captions = small_lstm_model.sample(features)
    sample_captions = decode_captions(sample_captions, data['idx_to_word'])

    for gt_caption, sample_caption, url in zip(gt_captions, sample_captions, urls):
        img = image_from_url(url)
        # Skip missing URLs.
        if img is None: continue
        plt.imshow(img) 
        plt.title('%s\n%s\nGT:%s' % (split, sample_caption, gt_caption))
        plt.axis('off')
        plt.show()